In [ ]:
# Softmax classifier for guessing minesweeper board position that is mine-free

In [1]:
# Import libraries for simulation
import tensorflow as tf
import numpy as np
import random as r
import datetime as dt

/home/ruben/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
dimensions = (12,12)
mineProbability = 0.16      # Probability that a square contain a mine

In [3]:
# This is a matrix that maps mine board vectors to mine count vectors
def minesweepMatrix(dimensions):
    rows,cols = dimensions
    size = rows * cols
    A = np.zeros([size,size],dtype=int)
    for rA in range(size):
        for cA in range(size):
            inRow, inCol = divmod(rA,cols)
            outRow, outCol = divmod(cA,cols)
            A[rA,cA] = abs(inRow-outRow) <= 1 and abs(inCol-outCol) <= 1
    return(A)

In [4]:
# Converts a board of mines into a board of mine counts
def boardMineCounts(board):
    return(minesweepMatrix(board.shape).dot(board.flatten()).reshape(board.shape))

In [5]:
# This takes a mine board and gives a mine count with mines removed, and other random squares removed
def boardPartialMineCounts(board):
    missingProbability = r.uniform(0.05,0.8)
    result = boardMineCounts(board)
    for index, x in np.ndenumerate(board):
        if x: result[index] = -1
        elif r.uniform(0, 1) < missingProbability: result[index] = -1
    return result

In [6]:
# Generates a random training batch of size at most n
def next_training_batch(n):
    batch_xs = []
    batch_ys = []
    for _ in range(n):
        board = np.random.random(dimensions) < mineProbability
        counts = boardPartialMineCounts(board)
        frees = ((counts == -1).astype(int) - board).flatten().astype(float)    #(1 - board).flatten().astype(float)
        freesSum = sum(frees)
        if freesSum > 0:
            batch_xs.append(counts.flatten())
            batch_ys.append(frees / freesSum)
    return (np.asarray(batch_xs), np.asarray(batch_ys))

In [8]:
# Create the model
rows, cols = dimensions
size = rows*cols
mineCounts = tf.placeholder(tf.int32, [None, size], name="mineCounts")
mineCountsOneHot = tf.reshape(tf.one_hot(mineCounts+1,10), [-1, size*10])
W = tf.Variable(tf.random_normal([size*10, size], stddev=0.01), name="W")
b = tf.Variable(tf.random_normal([size], stddev=0.01), name="b")
y = tf.matmul(mineCountsOneHot, W) + b

In [9]:
mineFreeAverages = tf.placeholder(tf.float32, [None, size], name="mineFreeAverages")

In [10]:
# Loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=mineFreeAverages, logits=y))

In [11]:
# Summaries for tensorboard
with tf.name_scope('W_reshape'):
    image_shaped_W = tf.reshape(W, [-1, size*10, size, 1])
    tf.summary.image('W', image_shaped_W, 1000)

with tf.name_scope('b_reshape'):
    image_shaped_b = tf.reshape(b, [-1, rows, cols, 1])
    tf.summary.image('b', image_shaped_b, 1000)

_ = tf.summary.scalar('accuracy', cross_entropy)

In [12]:
# Optimiser
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

In [13]:
# Create session and initialise or restore stuff
savePath = './saves.tf.Mines4/' + str(dimensions) + '/'
saver = tf.train.Saver()

sess = tf.InteractiveSession()

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('.', sess.graph)

In [14]:
tf.global_variables_initializer().run()

In [15]:
# Restore model?
#saver.restore(sess, savePath + "model-10000")

In [16]:
# Train
for iteration in range(10001):
    batch_xs, batch_ys = next_training_batch(100)
    if iteration % 10 == 0:
        summary, loss, _ = sess.run([merged, cross_entropy, train_step],
                                   feed_dict={mineCounts: batch_xs, mineFreeAverages: batch_ys})
        writer.add_summary(summary, iteration)
        print('%s: Loss at step %s: %s' % (dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), iteration, loss))
    else:
        _ = sess.run(train_step, feed_dict={mineCounts: batch_xs, mineFreeAverages: batch_ys})
    if iteration % 1000 == 0:
        save_path = saver.save(sess, savePath + 'model', global_step=iteration)
        print("Model saved in file: %s" % save_path)

2017-11-06 09:47:57: Loss at step 0: 4.97615
Model saved in file: ./saves.tf.Mines4/(12, 12)/model-0
2017-11-06 09:48:03: Loss at step 10: 4.96654
2017-11-06 09:48:10: Loss at step 20: 4.95619
2017-11-06 09:48:17: Loss at step 30: 4.95212
2017-11-06 09:48:23: Loss at step 40: 4.94011
2017-11-06 09:48:30: Loss at step 50: 4.93051
2017-11-06 09:48:36: Loss at step 60: 4.92285
2017-11-06 09:48:43: Loss at step 70: 4.91178
2017-11-06 09:48:49: Loss at step 80: 4.89853
2017-11-06 09:48:56: Loss at step 90: 4.89784
2017-11-06 09:49:03: Loss at step 100: 4.88127
2017-11-06 09:49:09: Loss at step 110: 4.87443
2017-11-06 09:49:16: Loss at step 120: 4.86731
2017-11-06 09:49:22: Loss at step 130: 4.85594
2017-11-06 09:49:29: Loss at step 140: 4.84375
2017-11-06 09:49:35: Loss at step 150: 4.83709
2017-11-06 09:49:42: Loss at step 160: 4.83293
2017-11-06 09:49:48: Loss at step 170: 4.83216
2017-11-06 09:49:55: Loss at step 180: 4.79398
2017-11-06 09:50:01: Loss at step 190: 4.82019
2017-11-06 09:5

2017-11-06 10:06:41: Loss at step 1710: 4.21025
2017-11-06 10:06:47: Loss at step 1720: 4.28632
2017-11-06 10:06:54: Loss at step 1730: 4.29179
2017-11-06 10:07:01: Loss at step 1740: 4.19269
2017-11-06 10:07:07: Loss at step 1750: 4.23707
2017-11-06 10:07:14: Loss at step 1760: 4.23297
2017-11-06 10:07:20: Loss at step 1770: 4.21388
2017-11-06 10:07:27: Loss at step 1780: 4.22434
2017-11-06 10:07:33: Loss at step 1790: 4.17666
2017-11-06 10:07:40: Loss at step 1800: 4.18607
2017-11-06 10:07:47: Loss at step 1810: 4.17475
2017-11-06 10:07:53: Loss at step 1820: 4.1837
2017-11-06 10:08:00: Loss at step 1830: 4.24967
2017-11-06 10:08:06: Loss at step 1840: 4.26028
2017-11-06 10:08:13: Loss at step 1850: 4.22317
2017-11-06 10:08:19: Loss at step 1860: 4.26801
2017-11-06 10:08:26: Loss at step 1870: 4.22046
2017-11-06 10:08:33: Loss at step 1880: 4.24806
2017-11-06 10:08:39: Loss at step 1890: 4.23694
2017-11-06 10:08:46: Loss at step 1900: 4.10268
2017-11-06 10:08:52: Loss at step 1910: 4

2017-11-06 10:25:12: Loss at step 3400: 4.23046
2017-11-06 10:25:19: Loss at step 3410: 4.04878
2017-11-06 10:25:26: Loss at step 3420: 4.03863
2017-11-06 10:25:32: Loss at step 3430: 4.09786
2017-11-06 10:25:39: Loss at step 3440: 4.13433
2017-11-06 10:25:45: Loss at step 3450: 4.09914
2017-11-06 10:25:52: Loss at step 3460: 4.09835
2017-11-06 10:25:58: Loss at step 3470: 4.12781
2017-11-06 10:26:05: Loss at step 3480: 4.10683
2017-11-06 10:26:12: Loss at step 3490: 4.07587
2017-11-06 10:26:18: Loss at step 3500: 4.10672
2017-11-06 10:26:25: Loss at step 3510: 4.04751
2017-11-06 10:26:31: Loss at step 3520: 4.14033
2017-11-06 10:26:38: Loss at step 3530: 4.09707
2017-11-06 10:26:44: Loss at step 3540: 4.05078
2017-11-06 10:26:51: Loss at step 3550: 4.09735
2017-11-06 10:26:58: Loss at step 3560: 4.03425
2017-11-06 10:27:04: Loss at step 3570: 4.09411
2017-11-06 10:27:11: Loss at step 3580: 4.03119
2017-11-06 10:27:18: Loss at step 3590: 4.17572
2017-11-06 10:27:25: Loss at step 3600: 

2017-11-06 10:43:46: Loss at step 5090: 4.08018
2017-11-06 10:43:52: Loss at step 5100: 3.97637
2017-11-06 10:43:59: Loss at step 5110: 4.12944
2017-11-06 10:44:05: Loss at step 5120: 4.09478
2017-11-06 10:44:12: Loss at step 5130: 4.13756
2017-11-06 10:44:19: Loss at step 5140: 4.06988
2017-11-06 10:44:25: Loss at step 5150: 4.09276
2017-11-06 10:44:32: Loss at step 5160: 4.00183
2017-11-06 10:44:38: Loss at step 5170: 3.99101
2017-11-06 10:44:45: Loss at step 5180: 4.12111
2017-11-06 10:44:51: Loss at step 5190: 4.04455
2017-11-06 10:44:58: Loss at step 5200: 4.08476
2017-11-06 10:45:05: Loss at step 5210: 4.09026
2017-11-06 10:45:11: Loss at step 5220: 4.05058
2017-11-06 10:45:18: Loss at step 5230: 4.1
2017-11-06 10:45:24: Loss at step 5240: 4.06581
2017-11-06 10:45:31: Loss at step 5250: 4.14531
2017-11-06 10:45:37: Loss at step 5260: 4.06103
2017-11-06 10:45:44: Loss at step 5270: 4.09071
2017-11-06 10:45:51: Loss at step 5280: 4.19195
2017-11-06 10:45:57: Loss at step 5290: 4.23

2017-11-06 11:02:27: Loss at step 6790: 4.12899
2017-11-06 11:02:33: Loss at step 6800: 4.01099
2017-11-06 11:02:40: Loss at step 6810: 4.07888
2017-11-06 11:02:47: Loss at step 6820: 3.9977
2017-11-06 11:02:54: Loss at step 6830: 4.02959
2017-11-06 11:03:01: Loss at step 6840: 3.98307
2017-11-06 11:03:07: Loss at step 6850: 4.06857
2017-11-06 11:03:14: Loss at step 6860: 4.08125
2017-11-06 11:03:21: Loss at step 6870: 3.97359
2017-11-06 11:03:27: Loss at step 6880: 4.07388
2017-11-06 11:03:34: Loss at step 6890: 4.0173
2017-11-06 11:03:41: Loss at step 6900: 4.11419
2017-11-06 11:03:48: Loss at step 6910: 4.12847
2017-11-06 11:03:55: Loss at step 6920: 4.10905
2017-11-06 11:04:01: Loss at step 6930: 4.0322
2017-11-06 11:04:09: Loss at step 6940: 4.04721
2017-11-06 11:04:15: Loss at step 6950: 3.96631
2017-11-06 11:04:22: Loss at step 6960: 4.12861
2017-11-06 11:04:29: Loss at step 6970: 4.02876
2017-11-06 11:04:35: Loss at step 6980: 4.08305
2017-11-06 11:04:42: Loss at step 6990: 4.1

2017-11-06 11:21:13: Loss at step 8480: 4.10959
2017-11-06 11:21:20: Loss at step 8490: 4.00409
2017-11-06 11:21:26: Loss at step 8500: 4.05166
2017-11-06 11:21:33: Loss at step 8510: 4.0291
2017-11-06 11:21:40: Loss at step 8520: 4.01697
2017-11-06 11:21:46: Loss at step 8530: 4.0703
2017-11-06 11:21:53: Loss at step 8540: 4.02369
2017-11-06 11:22:00: Loss at step 8550: 3.98684
2017-11-06 11:22:07: Loss at step 8560: 4.08523
2017-11-06 11:22:13: Loss at step 8570: 4.01986
2017-11-06 11:22:20: Loss at step 8580: 4.08698
2017-11-06 11:22:27: Loss at step 8590: 4.20879
2017-11-06 11:22:33: Loss at step 8600: 4.04064
2017-11-06 11:22:40: Loss at step 8610: 4.08969
2017-11-06 11:22:46: Loss at step 8620: 4.05002
2017-11-06 11:22:53: Loss at step 8630: 4.21123
2017-11-06 11:23:00: Loss at step 8640: 4.14405
2017-11-06 11:23:06: Loss at step 8650: 4.08141
2017-11-06 11:23:13: Loss at step 8660: 4.11447
2017-11-06 11:23:19: Loss at step 8670: 4.06205
2017-11-06 11:23:26: Loss at step 8680: 4.

In [17]:
# Test trained model on larger batch size
batch_xs, batch_ys = next_training_batch(1000)
print(sess.run(cross_entropy, feed_dict={mineCounts: batch_xs, mineFreeAverages: batch_ys}))

4.07932


In [25]:
# Run a test
batchSize = 10000
batch_xs, batch_ys = next_training_batch(batchSize)

predictions = sess.run(tf.nn.softmax(y), feed_dict={mineCounts: batch_xs, mineFreeAverages: batch_ys})
bestSquares = [pred.argmax() for pred in predictions]
unfrees = (batch_ys == 0).astype(int)
frees = [unfrees[i][bestSquares[i]] for i in range(batchSize)]
print("Number of errors for batch size of ", batchSize)
print(sum(frees))

Number of errors for batch size of  10000
166


In [28]:
# Find boards that we failed on
batchSize = 1000
batch_xs, batch_ys = next_training_batch(batchSize)

predictions = sess.run(tf.nn.softmax(y), feed_dict={mineCounts: batch_xs, mineFreeAverages: batch_ys})
bestSquares = [pred.argmax() for pred in predictions]
unfrees = (batch_ys == 0).astype(int)
guesses = [unfrees[i][bestSquares[i]] for i in range(batchSize)]
for i in range(batchSize):
    if guesses[i] == 1:
        print(batch_xs[i].reshape(dimensions))
        summary = sess.run(tf.summary.image('mine_miss', tf.reshape((batch_xs[i]+1).astype(float),[-1,rows,cols,1]), 100))
        writer.add_summary(summary)

[[-1 -1  1  0 -1  1 -1  1  1 -1  1 -1]
 [-1  3  3 -1 -1  2 -1  3 -1  3  2  1]
 [ 1 -1 -1  2 -1  2 -1 -1 -1  2 -1  1]
 [-1 -1 -1  2  0  1  2 -1 -1  3 -1  2]
 [-1  2  1 -1  1  1  1 -1  1  1 -1  2]
 [-1  1  0  1 -1  2  1 -1  1 -1 -1 -1]
 [ 2  2 -1  1 -1 -1  1  0  1 -1  3 -1]
 [-1  2 -1  0 -1 -1  2  2  4  3 -1  1]
 [-1 -1  1  0  0 -1  2 -1 -1 -1  1  0]
 [-1 -1  1  0 -1  0  2 -1  5  3 -1 -1]
 [-1 -1 -1  0  0  1  3  3  3 -1 -1 -1]
 [ 2 -1  1  0  0  1 -1 -1 -1 -1  1  0]]
[[ 1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1  2  2 -1 -1]
 [-1 -1 -1 -1  5 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1  4 -1 -1 -1 -1 -1 -1  3]
 [-1 -1 -1 -1  2 -1 -1 -1 -1  2 -1  2]
 [-1 -1 -1 -1  1 -1 -1 -1 -1  2 -1 -1]
 [ 1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1  2 -1 -1 -1 -1  2 -1  2]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1  1 -1  1 -1 -1 -1 -1 -1  1 -1 -1]
 [-1 -1  0 -1 -1  0 -1 -1 -1 -1 -1  0]
 [ 0  0  0 -1 -1 -1 -1 -1 -1 -1 -1 -1]]
[[-1  3  1  1  0  0  1  2 -1 -1  0  0]
 [-1  4 -1  1  0  0  2 